In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
import utils
import webdataset as wds
import os
import pandas as pd
from torch.utils.data import Dataset

/home/dongho/anaconda3/envs/temp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # load package
# import numpy as np
# import scipy.io

# file_name_nsd_stim = './nsd_stim_info_merged.csv'
# file_name_emotic_annot = './emotic_annotations.mat'

# ## get EMOTIC data
# data = scipy.io.loadmat(file_name_emotic_annot, simplify_cells=True)
# emotic_data = data['train'] + data['test'] + data['val']
# emotic_coco_data = [x for x in emotic_data if x['original_database']['name']=='mscoco']
# coco_id = [x['original_database']['info']['image_id'] for x in emotic_coco_data]
# annotations = [x['person'] for x in emotic_coco_data] 
# emotic_annotations = []
# for annot in annotations:
#     annot = [annot] if type(annot)==dict else annot

#     valence = []; arousal = []; dominance = []
#     for person in annot:
#         person = person['annotations_continuous']
#         person = [person] if type(person)==dict else person
#         valence += [np.mean([x['valence'] for x in person])]
#         arousal += [np.mean([x['arousal'] for x in person])]
#         dominance += [np.mean([x['dominance'] for x in person])]
#     emotic_annotations += [{ 'valence':valence, 'arousal':arousal, 'dominance':dominance}]

# emotic_annotations = dict(zip(coco_id, emotic_annotations))

# # define function
# get_emotic_annot       = lambda coco_id, metric_type: np.mean(emotic_annotations[coco_id][metric_type])
# get_emotic_annot_indiv = lambda coco_id, metric_type: emotic_annotations[coco_id][metric_type]

# ## get NSD data
# df = data = pd.read_csv(file_name_nsd_stim)
# nsd_id = df['Unnamed: 0'].values
# nsd_cocoid = df['cocoId'].values
# nsd_cocosplit = df['cocoSplit'].values
# nsd_isshared = df['shared1000'].values


# ## target NSD data
# joint_cocoid = nsd_cocoid[np.isin(nsd_cocoid, list(emotic_annotations.keys()))]
# # print(target_cocoid)
# print("The Num of intersection between NSD and EMOTIC dataSet:", len(joint_cocoid))

# """
# 1. TrainSet: NSD 데이터셋에서 개별적으로 보여준 것
# 2. TestSet: NSD 데이터셋에서 공통으로 보여준 것
# 3. 공통으로, 사람 한명 있는 데이터 사용
#  - `len(person)==1`
#  """
# # 사람 한명 있는 데이터
# target_cocoid = [coco_id for coco_id in joint_cocoid if len(get_emotic_annot_indiv(coco_id, 'valence')) == 1]
# print("The number of joint pictures which reveals only one person:", len(target_cocoid))
# train_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  ~nsd_isshared]
# print(f"cocoid for train, #: {len(train_cocoid)}")
# test_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  nsd_isshared]
# print(f"cocoid for test, #: {len(test_cocoid)}")

# # data for common image shown to all subjects
# # regarding as test set
# test_df = df[df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]
# train_df =  df[~df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]

# for i in range(1, 9):
#     current_subject = f"subject{i}"
#     curr_df = train_df[train_df[current_subject] == True] # filter only shown to curr_subject

#     # 현재 subject의 데이터에서 train_cocoid와 겹치는 데이터 찾기
#     # 즉 현재 subject에 보여준 사진 중 cocoid가 EMOTIC dataset에 사용되었던 것 필터링
#     target_train_df = curr_df[np.isin(curr_df["cocoId"], train_cocoid)]
#     print(f"data of {current_subject}: {target_train_df.shape}")

# target_test_df = test_df[np.isin(test_df["cocoId"], test_cocoid)]

In [3]:
def reshape_brain3d(brain_3d, target_shape=(96, 96, 96)):
    # Initialize pad_width
    pad_width = [(0, 0)] * 3  # For 3D brain_3day

    # Calculate padding needed for each dimension
    for i in range(3):
        current_size = brain_3d.shape[i]
        if current_size < target_shape[i]:
            # Calculate padding
            total_pad = target_shape[i] - current_size
            pad_before = total_pad // 2
            pad_after = total_pad - pad_before
            pad_width[i] = (pad_before, pad_after)

    # Apply padding
    brain_3d = np.pad(brain_3d, pad_width=pad_width, mode='constant', constant_values=0)

    # Apply truncation if necessary
    brain_3d = brain_3d[:target_shape[0], :target_shape[1], :target_shape[2]]

    # print("reshaped brain_3d shape", brain_3d.shape)
    return brain_3d

In [4]:
# import scipy

# class BrainValenceDataset(Dataset):
#     def __init__(self, data_path, split, subj):
#         self.data_path = data_path
#         self.split = split # train, val, test
#         self.subj = subj
#         self.metadata = pd.read_csv(os.path.join(self.data_path, f'{self.split}_subj0{self.subj}_metadata.csv'))

#         ## get joint data between NSD and EMOTIC and COCO
#         self.emotic_annotations = self.get_emotic_data()
#         self.target_cocoid = self.get_NSD_data(self.emotic_annotations)

#         ## use only joint and target(one person in picture) image
#         self.valid_indices = []
#         for idx in range(len(self.metadata)):
#             sample = self.metadata.iloc[idx]
#             cocoid = torch.from_numpy(np.load(os.path.join(self.data_path, self.split, sample['coco'])))
#             if cocoid in self.target_cocoid:
#                 self.valid_indices.append(idx)

#     def __len__(self):
#         return len(self.valid_indices)

#     def __getitem__(self, idx):
#         actual_idx = self.valid_indices[idx]

#         sample = self.metadata.iloc[actual_idx]
#         cocoid = torch.from_numpy(np.load(os.path.join(self.data_path, self.split, sample['coco'])))
#         brain_3d = torch.from_numpy(np.load(os.path.join(self.data_path, self.split, sample['mri']))) # (3, *, *, *)
#         brain_3d = np.mean(brain_3d, axis=0) # (*, *, *)
#         brain_3d = self.reshape_brain3d(brain_3d, target_shape=(96, 96, 96)) # (96, 96, 96)
        
#         valence = np.mean(self.emotic_annotations[cocoid]['valence'])

#         return cocoid, brain_3d, valence
    
#     def reshape_brain3d(brain_3d, target_shape=(96, 96, 96)):
#         # Initialize pad_width
#         pad_width = [(0, 0)] * 3  # For 3D brain_3day

#         # Calculate padding needed for each dimension
#         for i in range(3):
#             current_size = brain_3d.shape[i]
#             if current_size < target_shape[i]:
#                 # Calculate padding
#                 total_pad = target_shape[i] - current_size
#                 pad_before = total_pad // 2
#                 pad_after = total_pad - pad_before
#                 pad_width[i] = (pad_before, pad_after)

#         # Apply padding
#         brain_3d = np.pad(brain_3d, pad_width=pad_width, mode='constant', constant_values=0)

#         # Apply truncation if necessary
#         brain_3d = brain_3d[:target_shape[0], :target_shape[1], :target_shape[2]]

#         return brain_3d
    
#     def get_emotic_data() -> dict:
#         file_name_emotic_annot = './emotic_annotations.mat'

#         ## get EMOTIC data
#         data = scipy.io.loadmat(file_name_emotic_annot, simplify_cells=True)
#         emotic_data = data['train'] + data['test'] + data['val']
#         emotic_coco_data = [x for x in emotic_data if x['original_database']['name']=='mscoco']
#         coco_id = [x['original_database']['info']['image_id'] for x in emotic_coco_data]
#         annotations = [x['person'] for x in emotic_coco_data] 
#         emotic_annotations = []
#         for annot in annotations:
#             annot = [annot] if type(annot)==dict else annot

#             valence = []; arousal = []; dominance = []
#             for person in annot:
#                 person = person['annotations_continuous']
#                 person = [person] if type(person)==dict else person
#                 valence += [np.mean([x['valence'] for x in person])]
#                 arousal += [np.mean([x['arousal'] for x in person])]
#                 dominance += [np.mean([x['dominance'] for x in person])]
#             emotic_annotations += [{ 'valence':valence, 'arousal':arousal, 'dominance':dominance}]

#         emotic_annotations = dict(zip(coco_id, emotic_annotations))

#         return emotic_annotations

#     def get_NSD_data(emotic_annotations):
#         # out: target_cocoid
#         file_name_nsd_stim = './nsd_stim_info_merged.csv'

#         ## get NSD data
#         df = data = pd.read_csv(file_name_nsd_stim)
#         nsd_id = df['Unnamed: 0'].values
#         nsd_cocoid = df['cocoId'].values
#         nsd_cocosplit = df['cocoSplit'].values
#         nsd_isshared = df['shared1000'].values

#         joint_cocoid = nsd_cocoid[np.isin(nsd_cocoid, list(emotic_annotations.keys()))]
#         target_cocoid = [coco_id for coco_id in joint_cocoid if len(emotic_annotations[coco_id]['valence']) == 1]
#         train_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  ~nsd_isshared]
#         test_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  nsd_isshared]

#         # data for common image shown to all subjects
#         # regarding as test set
#         # test_df = df[df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]
#         # train_df = df[~df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]

#         return target_cocoid
    

In [36]:
import scipy
    
def get_emotic_data() -> dict:
    file_name_emotic_annot = './emotic_annotations.mat'

    ## get EMOTIC data
    data = scipy.io.loadmat(file_name_emotic_annot, simplify_cells=True)
    emotic_data = data['train'] + data['test'] + data['val']
    emotic_coco_data = [x for x in emotic_data if x['original_database']['name']=='mscoco']
    coco_id = [x['original_database']['info']['image_id'] for x in emotic_coco_data]
    annotations = [x['person'] for x in emotic_coco_data] 
    emotic_annotations = []
    for annot in annotations:
        annot = [annot] if type(annot)==dict else annot

        valence = []; arousal = []; dominance = []
        for person in annot:
            person = person['annotations_continuous']
            person = [person] if type(person)==dict else person
            valence += [np.mean([x['valence'] for x in person])]
            arousal += [np.mean([x['arousal'] for x in person])]
            dominance += [np.mean([x['dominance'] for x in person])]
        emotic_annotations += [{ 'valence':valence, 'arousal':arousal, 'dominance':dominance}]

    emotic_annotations = dict(zip(coco_id, emotic_annotations))

    return emotic_annotations

def get_NSD_data(emotic_annotations):
    # out: target_cocoid
    file_name_nsd_stim = './nsd_stim_info_merged.csv'

    ## get NSD data
    df = pd.read_csv(file_name_nsd_stim)
    nsd_id = df['nsdId'].values
    nsd_cocoid = df['cocoId'].values
    nsd_cocosplit = df['cocoSplit'].values
    nsd_isshared = df['shared1000'].values

    joint_cocoid = nsd_cocoid[np.isin(nsd_cocoid, list(emotic_annotations.keys()))]
    target_cocoid = [coco_id for coco_id in joint_cocoid if len(emotic_annotations[coco_id]['valence']) == 1]
    train_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  ~nsd_isshared]
    test_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  nsd_isshared]

    return df, target_cocoid
    

In [37]:
emotic_annotations = get_emotic_data()
df, target_cocoid = get_NSD_data(emotic_annotations)

### Cautions
- metadata에 해당하는 data의 coco가 coco_id를 말하는 것이 아니라, 
- `nsd_stim_info_merged.csv` 파일의 인덱스를 뜻하는 거였음.....
- 그래서 coco -> coco_id 변환용 Dataframe 및 함수가 필요함.

In [39]:
data_path="/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split"
split="val"
subjects=[1, 2, 5, 7]

dfs = [pd.read_csv(os.path.join(data_path, f'{split}_subj0{subj}_metadata.csv')) for subj in subjects]
metadata = pd.concat(dfs)
metadata.reset_index(inplace=True, drop=True)
print(len(metadata))

nsd_id = metadata['coco'].apply(lambda x: np.load(data_path+f"/{split}/"+x)[-1])
# coco_id = df[df['nsdId'].isin(nsd_id)]['cocoId']
coco_id = []
isin = nsd_id.isin(df['nsdId'])#; print(isin)
for i, x in nsd_id.items():
    if isin[i]: coco_id.append(df.loc[x, 'cocoId'])

coco_id = pd.Series(coco_id)

1200


In [57]:
class BrainValenceDataset(Dataset):
    def __init__(self,  data_path, split, emotic_annotations, nsd_df, target_cocoid, subjects=[1, 2, 5, 7]):
        self.data_path = data_path
        self.split = split # train, val, test
        self.subjects = subjects # [1, 2, 5, 7]
        if split in ['train', 'val']:
            dfs = [pd.read_csv(os.path.join(self.data_path, f'{self.split}_subj0{subj}_metadata.csv')) for subj in self.subjects]
            self.metadata = pd.concat(dfs)
        elif split == 'test':
            # As test data is all same to every subjects,
            df = pd.read_csv(os.path.join(self.data_path, 'test_subj01_metadata.csv'))
            self.metadata = df
        else: IndexError("Wrong split type")
        
        self.metadata.reset_index(inplace=True, drop=True)       

        ## get joint data between NSD and EMOTIC and COCO
        self.nsd_df = nsd_df
        self.emotic_annotations = emotic_annotations
        self.target_cocoid = target_cocoid

        # pre convert nsd data appropriately
        self.coco_id = self.nsd2coco()

        ## use only joint and target(one person in picture) image
        # target_cocoid와 joint 한 것만 남긴다.
        isin = self.coco_id.isin(self.target_cocoid)
        self.coco_id = self.coco_id[isin]
        # metadata도 남길 애들만 남긴다.
        self.metadata = self.metadata[isin]
        

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):

        sample = self.metadata.iloc[idx]
        id = self.coco_id.iloc[idx]
        brain_3d = torch.from_numpy(np.load(os.path.join(self.data_path, self.split, sample['mri']))) # (3, *, *, *)
        brain_3d = np.mean(brain_3d, axis=0) # (*, *, *)
        brain_3d = self.reshape_brain3d(brain_3d, target_shape=(96, 96, 96)) # (96, 96, 96)
        
        valence = np.mean(self.emotic_annotations[id]['valence'])

        return brain_3d, valence

    def nsd2coco(self) -> pd.Series :
        # metadata.csv의 coco column은 nsd_id이므로, 이를 `nsd_stim_info_merged.csv` 를 읽어온 후, 
        # nsdid => coco id 로 바꿔줌

        nsd_id = self.metadata['coco'].apply(lambda x: np.load(self.data_path+f"/{self.split}/"+x)[-1])
        coco_id = []
        isin = nsd_id.isin(self.nsd_df['nsdId'])
        for i, x in nsd_id.items():
            if isin[i]: coco_id.append(self.nsd_df.loc[x, 'cocoId'])

        coco_id = pd.Series(coco_id)

        return coco_id
    
    def reshape_brain3d(brain_3d, target_shape=(96, 96, 96)):
        # Initialize pad_width
        pad_width = [(0, 0)] * 3  # For 3D brain_3day

        # Calculate padding needed for each dimension
        for i in range(3):
            current_size = brain_3d.shape[i]
            if current_size < target_shape[i]:
                # Calculate padding
                total_pad = target_shape[i] - current_size
                pad_before = total_pad // 2
                pad_after = total_pad - pad_before
                pad_width[i] = (pad_before, pad_after)

        # Apply padding
        brain_3d = np.pad(brain_3d, pad_width=pad_width, mode='constant', constant_values=0)

        # Apply truncation if necessary
        brain_3d = brain_3d[:target_shape[0], :target_shape[1], :target_shape[2]]

        return brain_3d

In [54]:
train_dataset = BrainValenceDataset(
    data_path="/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split",
    split="train",
    emotic_annotations=emotic_annotations,
    nsd_df=df,
    target_cocoid=target_cocoid,
    subjects=[1, 2, 5, 7]
)

len(train_dataset)

3582

In [55]:
val_dataset = BrainValenceDataset(
    data_path="/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split",
    split="val",
    emotic_annotations=emotic_annotations,
    nsd_df=df,
    target_cocoid=target_cocoid,
    subjects=[1, 2, 5, 7]
)

len(val_dataset)

88

In [58]:
test_dataset = BrainValenceDataset(
    data_path="/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split",
    split="test",
    emotic_annotations=emotic_annotations,
    nsd_df=df,
    target_cocoid=target_cocoid,
    subjects=[1, 2, 5, 7]
)

len(test_dataset)

110